# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dingcheng,32.1272,115.0394,14.97,79,0,3.43,CN,1731276982
1,1,crane,31.3974,-102.3501,23.82,12,0,3.08,US,1731276984
2,2,aguas vermelhas,-15.7472,-41.4600,19.22,97,96,2.46,BR,1731276985
3,3,betio village,1.3580,172.9211,31.02,73,78,7.90,KI,1731276986
4,4,cabo san lucas,22.8909,-109.9124,26.17,65,0,2.57,MX,1731276987


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
pip install cartopy geoviews pyproj


Note: you may need to restart the kernel to use updated packages.


In [23]:
import hvplot.pandas  # Make sure to import hvplot

# Configure the map
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    tiles="OSM", 
    frame_width=700, 
    frame_height=500
)

# Display the map
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit the criteria (for example, Northern Hemisphere cities)
northern_hemisphere_df = city_data_df[city_data_df['Lat'] >= 0].copy()

# Drop any rows with null values
northern_hemisphere_df.dropna(inplace=True)

# Display sample data
northern_hemisphere_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dingcheng,32.1272,115.0394,14.97,79,0,3.43,CN,1731276982
1,1,crane,31.3974,-102.3501,23.82,12,0,3.08,US,1731276984
3,3,betio village,1.3580,172.9211,31.02,73,78,7.90,KI,1731276986
4,4,cabo san lucas,22.8909,-109.9124,26.17,65,0,2.57,MX,1731276987
5,5,whitehorse,60.7161,-135.0538,0.01,74,100,5.66,CA,1731276988


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Create a copy of the DataFrame to store city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column for "Hotel Name"
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,dingcheng,CN,32.1272,115.0394,79,
1,crane,US,31.3974,-102.3501,12,
2,aguas vermelhas,BR,-15.7472,-41.4600,97,
3,betio village,KI,1.3580,172.9211,73,
4,cabo san lucas,MX,22.8909,-109.9124,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 1000  # Search within a 1000m radius (you can adjust this value)
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1,  # Limit results to 1 hotel
    "apiKey": "1d26ab9d68a648c4ad59a007d2d12a96"  # Replace with your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["lat"] = lat
    params["lon"] = lng

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
dingcheng - nearest hotel: 交通宾馆
crane - nearest hotel: Mainstay Suites
aguas vermelhas - nearest hotel: No hotel found
betio village - nearest hotel: Seaman Hostel
cabo san lucas - nearest hotel: Comfort Rooms
whitehorse - nearest hotel: Town & Mountain Hotel
qaqortoq - nearest hotel: Hotel Qaqortoq
port elizabeth - nearest hotel: Waterford Hotel
vilyuysk - nearest hotel: Мария
port-aux-francais - nearest hotel: Keravel
hermanus - nearest hotel: Aloe guest house
bilibino - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
turkmenabat - nearest hotel: Yupek Yoly
olonkinbyen - nearest hotel: Hótel Norðurljós
waitangi - nearest hotel: Hotel Chathams
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
atafu village - nearest hotel: Luana Liki Hotel
iskateley - nearest hotel: Авантаж
longyearbyen - nearest hotel: The Vault
grytviken - nearest hotel: Shorty's Motel
yellowknife - nearest hotel: No hotel found
daru - nearest hotel: Kuki
tingi - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,dingcheng,CN,32.1272,115.0394,79,交通宾馆
1,crane,US,31.3974,-102.3501,12,Mainstay Suites
2,aguas vermelhas,BR,-15.7472,-41.4600,97,No hotel found
3,betio village,KI,1.3580,172.9211,73,Seaman Hostel
4,cabo san lucas,MX,22.8909,-109.9124,65,Comfort Rooms


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriImagery",
    color="City",
    size="Distance",  # Ensure this column is valid
    scale=0.1,  # Adjust scale if needed
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)